# 2-clean&filter

In [22]:
import pandas as pd

df = pd.read_csv(
    "../data/interim/extract_16.csv", low_memory=False, dtype={"ID_orateur": str}
)
df.shape

(337030, 32)

In [23]:
# Ne garder que les prises de parole et les interruptions
df = df[
    (df["Code_grammaire"] == "PAROLE_GENERIQUE")
    | (df["Code_grammaire"] == "INTERRUPTION_1_10")
]
# ou : df["Code_grammaire"].str.strip().isin(["PAROLE_GENERIQUE", "INTERRUPTION_1_10"])

In [24]:
df.shape

(274698, 32)

In [25]:
# Exclure les prises de parole de "Mme la présidente" et "M. le président"
# la col Role_debat n'est pas bien identifié en fait
# plutôt partir sur Nom_orateur

df = df[~df["Nom_orateur"].str.strip().isin(["M. le président", "Mme la présidente"])]

In [26]:
df.shape

(215306, 32)

In [27]:
# Garder une trace de la longueur des interventions
df["len_dirtytext"] = df["Texte"].str.len()

-> Finalement, on ne vire pas ici les types de code parole : on laisse faire la regex d'identification république pour faire le tri et on conserve ce qui est concerné quel que soit le type (= va exclure pas mal des catégories chelous)

In [28]:
# Pour le sous cas de la 16 législature : nettoyer le fichier qui n'est pas au bon endroit
# = date de 2012
# df["Session"].value_counts()

# (df["UID"] == "CRSANR5L16S2021O1N144").sum()
# df = df[~df["Nom_orateur"].str.strip().isin(["M. le président", "Mme la présidente"])]
df = df[df["UID"] != "CRSANR5L16S2021O1N144"]

## Nettoyer des données

In [29]:
# stabiliser le ID_orateur pour etre au format AN (pour matcher données)
df["ID_orateur"] = "PA" + df["ID_orateur"]

In [30]:
# Changer les missing values pour non_précisé (majoritaire) dans Code_parole
df["Code_parole"] = df["Code_parole"].fillna("non_précisé")

In [31]:
# aperçu des répartitions
df.groupby("Code_parole", dropna=False)["len_dirtytext"].describe()

,count,mean,std,min,25%,50%,75%,max
Code_parole,,,,,,,,
AVIS_COM_1_10,1.0,1278.000000,NaN,1278.0,1278.0,1278.0,1278.00,1278.0
AVIS_COM_1_20,10951.0,389.245457,416.340444,4.0,96.0,273.0,538.00,5395.0
AVIS_GVT_1_20,9954.0,418.768033,581.874887,4.0,20.0,212.0,581.75,6481.0
PAROLE_1_1,1.0,32.000000,NaN,32.0,32.0,32.0,32.00,32.0
PAROLE_1_2,70542.0,818.220961,1050.016891,3.0,221.0,463.0,999.00,21406.0
non_précisé,123678.0,257.965313,570.433599,3.0,18.0,38.0,195.00,16991.0


In [32]:
df.shape

(215127, 33)

## Match députés

In [33]:
df_deputes = pd.read_csv("../data/raw/id-dep/deputes-historique(datan-datagouv).csv")

In [34]:
# shape avant fusion
df.shape

(215127, 33)

In [35]:
# Join des données sur la base de l'id des intervenants
df = df.join(df_deputes.set_index("id"), on="ID_orateur", how="left", rsuffix="_dep")

In [36]:
# shape après fusion
df.shape

(215127, 59)

In [37]:
df.head()

,UID,SeanceRef,SessionRef,DateSeance,DateSeanceJour,NumSeanceJour,NumSeance,TypeAssemblee,Legislature,Session,...,facebook,website,nombreMandats,experienceDepute,scoreParticipation,scoreParticipationSpecialite,scoreLoyaute,scoreMajorite,active,dateMaj
4,CRSANR5L16S2023O1N173,RUANR5L16S2023IDS26877,SCR5A2023O1,20230313160000000,lundi 13 mars 2023,1,173,AN,16,Session ordinaire 2022-2023,...,AgnesPannierR,NaN,1.0,4 mois,0.00,NaN,NaN,NaN,0.0,2025-06-24
5,CRSANR5L16S2023O1N173,RUANR5L16S2023IDS26877,SCR5A2023O1,20230313160000000,lundi 13 mars 2023,1,173,AN,16,Session ordinaire 2022-2023,...,guillaumekasba,guillaumekasbarian.fr,3.0,7 ans,0.17,0.14,0.982,0.0,1.0,2025-06-24
6,CRSANR5L16S2023O1N173,RUANR5L16S2023IDS26877,SCR5A2023O1,20230313160000000,lundi 13 mars 2023,1,173,AN,16,Session ordinaire 2022-2023,...,AgnesPannierR,NaN,1.0,4 mois,0.00,NaN,NaN,NaN,0.0,2025-06-24
7,CRSANR5L16S2023O1N173,RUANR5L16S2023IDS26877,SCR5A2023O1,20230313160000000,lundi 13 mars 2023,1,173,AN,16,Session ordinaire 2022-2023,...,NaN,www.pierrecazeneuve.fr,2.0,3 ans,0.32,0.73,0.965,0.0,1.0,2025-06-24
8,CRSANR5L16S2023O1N173,RUANR5L16S2023IDS26877,SCR5A2023O1,20230313160000000,lundi 13 mars 2023,1,173,AN,16,Session ordinaire 2022-2023,...,AgnesPannierR,NaN,1.0,4 mois,0.00,NaN,NaN,NaN,0.0,2025-06-24


## Export

In [38]:
df.shape

(215127, 59)

In [39]:
# certaines col du join intrduisent une erreur à l'import/export
# forcer le QUOTE_ALL permet de résoudre

import csv  # pour résoudre le soucis d'écart. Checker

df.to_csv(
    "../data/interim/data_cleaning.csv",
    index=False,
    quoting=csv.QUOTE_ALL,  # a permis de résoudre le soucis d'écart. Checker
)

In [40]:
# verif excriture/lecture ok
df_test = pd.read_csv(
    "../data/interim/data_cleaning.csv", low_memory=False, dtype={"ID_orateur": str}
)
df_test.shape

(215127, 59)